<a href="https://colab.research.google.com/github/Zinkonaing-John/SLM/blob/master/SLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
training_data = [
   {"input": "The sky is ", "output": "blue."},
    {"input": "The flowers are", "output": "beautiful."},
    {"input": "Rain drops are", "output": "tiny."},
    {"input": "The stars are", "output": "twinkling."},
    {"input": "The sun is", "output": "shining."},
    {"input": "The moon is", "output": "illuminated."}
]

# Data တွေကို model နားလည်တဲ့ format အဖြစ် ပြောင်းလဲခြင်း
# ဒီနေရာမှာတော့ အလွယ်ကူဆုံးဖြစ်အောင် concept ကိုပဲ အရင်နားလည်စေချင်ပါတယ်
# တကယ် fine-tune လုပ်တဲ့အခါ ဒီထက်ပိုရှုပ်ထွေးတဲ့ data processing လိုအပ်ပါတယ်
print("Data ပြင်ဆင်ပြီးပါပြီ။")


Data ပြင်ဆင်ပြီးပါပြီ။


In [4]:
input_text = "The sky is"

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)

decoded_output = tokenizer.decode(outputs[0]),
skip_special_tokens = True

In [5]:

print(f"fine-tuning မလုပ်ခင် Model ရဲ့အဖြေ: {decoded_output}")

fine-tuning မလုပ်ခင် Model ရဲ့အဖြေ: ('<pad> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>',)


In [6]:
!pip install datasets

In [7]:
from datasets import Dataset

training_data_list =[
    {"input": "The sky is ", "output": "blue."},
    {"input": "The flowers are", "output": "beautiful."},
    {"input": "Rain drops are", "output": "tiny."},
    {"input": "The stars are", "output": "twinkling."},
    {"input": "The sun is", "output": "shining."},
    {"input": "The moon is", "output": "illuminated."}
]


training_dataset = Dataset.from_list(training_data_list)

def tokenize_function(examples):

  inputs = ["ကဗျာဖွဲ့ပါ: " + doc for doc in examples["input"]]

  model_inputs = tokenizer(inputs, max_length=128, truncation=True )
  labels = tokenizer(text_target=examples["output"],max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"] # Add this line to include labels

  return model_inputs

tokenized_dataset = training_dataset.map(tokenize_function, batched=True)

print("Dataset has been tokenized!")
print(tokenized_dataset)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Dataset has been tokenized!
Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 6
})


In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=50,
    per_device_train_batch_size=2,  # batch size per device during training
    learning_rate=0.001,

    logging_steps=10,
    # remove_unused_columns=False # Remove this line
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [9]:
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("Fine-tuning has started!")
trainer.train()
print("Fine-tuning has finished!")


Fine-tuning has started!


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: johnminnaing152200 (johnminnaing152200-jvision) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,1.454100
20,0.064700
30,0.001400
40,0.000500
50,0.000100
60,0.000300
70,0.000100
80,0.000100
90,0.000000
100,0.000100


Fine-tuning has finished!


In [12]:

# စမ်းသပ်မယ့် input စာသား
input_text = "The flowers are"

# input ကို model နားလည်တဲ့ format ဖြစ်အောင် ရှေ့မှာ instruction ထည့်ပေးပါ
prompt = "ကဗျာဖွဲ့ပါ: " + input_text

# input ကို tokenize လုပ်ခြင်း
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Fine-tune လုပ်ထားတဲ့ model ကို စာကြောင်းအသစ် generate လုပ်ခိုင်းခြင်း
outputs = model.generate(input_ids, max_new_tokens=20) # max_new_tokens နဲ့ စကားလုံးအရေအတွက်ကန့်သတ်နိုင်

# Generate လုပ်လို့ရလာတဲ့ IDs တွေကို လူနားလည်တဲ့ စာသားအဖြစ် ပြန်ပြောင်းခြင်း
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("---" * 10)
print(f"သင်ပေးလိုက်တဲ့ စာသား: {input_text}")
print(f"Fine-tuned Model ရဲ့အဖြေ: {decoded_output}")



------------------------------
သင်ပေးလိုက်တဲ့ စာသား: The flowers are
Fine-tuned Model ရဲ့အဖြေ: beautiful.
